In [ ]:
from llama_index.core import Settings 
from llama_index.llms.groq import Groq
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner
import arxiv
import os 

In [ ]:
llm = Groq(
    model="llama-3.3-70b-versatile",
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [ ]:
def calcular_imposto_renda(rendimento: float) -> str:
    if rendimento <= 22847.76:
        return "Isento de Imposto de Renda"
    elif rendimento <= 33919.80:
        imposto = (rendimento - 22847.76) * 0.075
    elif rendimento <= 45012.60:
        imposto = (rendimento - 33919.80) * 0.15 + 826.29
    elif rendimento <= 55976.16:
        imposto = (rendimento - 45012.60) * 0.225 + 1428.14
    else:
        imposto = (rendimento - 55976.16) * 0.275 + 2203.91

    return f"Imposto de Renda devido: R$ {imposto:.2f}"

In [ ]:
ferramenta_calculo_imposto = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="calcular_imposto_renda",
    description=(
        "Calcula o Imposto de Renda devido com base no rendimento anual. "
        "Argumento: rendimento (float) - Rendimento anual em reais. "
        "Retorna uma mensagem indicando se é isento ou o valor do imposto a pagar."
    ),

)

In [ ]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    llm=llm,
    tools=[ferramenta_calculo_imposto],
    verbose=True,
    allow_parallel_tool_calls=True, # Permite chamadas paralelas de ferramentas
)

In [ ]:
agent_imposto = AgentRunner(agent_worker_imposto)


In [ ]:
response = agent_imposto.chat(
    "Qual é o imposto de renda devido para um rendimento anual de R$ 50000.00?",
)

In [ ]:
def consulta_artigos(title: str) -> str:
    """
    Consulta artigos na base dados ArXiv e retorna resultados formatados.
    """
    # O arqiv é uma base de dados de artigos científicos
    buscar = arxiv.Search(
        query=title,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance,
    )

    resultados = [
        f"Título: {artigo.title}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in buscar.results()
    ]

    return "\n\n".join(resultados) if resultados else "Nenhum artigo encontrado."

In [ ]:
consulta_artigos_tool = FunctionTool.from_defaults(
    fn=consulta_artigos,
    name="consulta_artigos",
    description=(
        "Consulta artigos na base de dados ArXiv com base no título fornecido. "
        "Argumento: title (str) - Título do artigo a ser pesquisado. "
        "Retorna uma lista formatada de artigos encontrados."
    ),
)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    llm=llm,
    tools=[ferramenta_calculo_imposto, consulta_artigos_tool],
    verbose=True,
    # Desabilita chamadas paralelas de ferramentas, pois eu quero sequencialmente
    allow_parallel_tool_calls=False,  
)

In [ ]:
agent = AgentRunner(agent_worker)
response2 = agent.chat(
    "Quais artigos existem sobre 'Machine Learning'?",
)

In [ ]:
import os 
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
tavily_api_key = os.environ.get("TAVILY_API_KEY")


In [ ]:

from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(api_key=tavily_api_key) # type: ignore

In [65]:
tavily_tool_list = tavily_tool.to_tool_list()

for tool in tavily_tool_list:
    print(tool.metadata.name)

search


In [70]:
tavily_tool.search("Me retorne artigos sobre 'Machine Learning'", max_results=3) 

[Document(id_='ab0f530f-da01-474a-a6c5-c712ea554c74', embedding=None, metadata={'url': 'https://pmc.ncbi.nlm.nih.gov/articles/PMC8959062/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Este artigo de revisão aborda uma introdução sobre ML dividida em: definição, modelos de aprendizagem e uma revisão sistemática de artigos sobre a sua', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='dce4b9a4-5fa9-43f0-b0f4-d9ebb39c2445', embedding=None, metadata={'url': 'https://abccardiol.org/article/machine-learning-na-medicina-revisao-e-aplicabilidade/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddi